In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
import hook_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *
import csv

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [127]:
model = HookedTransformer.from_pretrained("gpt2-large")

with open('data/gpt2_large_spectrum_pre_v3.pkl', 'rb') as f:
    sample_df = pickle.load(f)

with open('data/gpt2_large_spectrum_pre_bins_v3.pkl', 'rb') as f:
    bin_edges = pickle.load(f)

def read_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return {row[0]: row[1] for row in csv.reader(f)}
    return {}

labels = read_labels('data/gp2_large_spectrum_pre_labels_v3.csv')
sorted_index_labels = sorted([(int(index), int(label)) for index, label in labels.items()])
sorted_labels = np.array([label for index, label in sorted_index_labels])

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


In [29]:
print(len(sorted_labels), len(sample_df))

1000 1000


In [128]:
sample_df['label'] = sorted_labels

In [31]:
text = load_dataset("stas/openwebtext-10k", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
grouped_total = sample_df.groupby('L31N892 Activation', as_index=False).count()['label']
grouped_3 = sample_df[sample_df['label'] == 3].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total
grouped_2 = sample_df[sample_df['label'] == 2].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total
grouped_1 = sample_df[sample_df['label'] == 1].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total

df = pd.DataFrame({
    'L31N892 Activation': grouped_3.index,
    'Next token is \' an\'' : grouped_3.values,
    'Next token is plausible \' an\'': grouped_2.values,
    'Next token is not \' an\'': grouped_1.values
})

In [49]:
grouped_melted = df.melt(id_vars=['L31N892 Activation'], 
                        value_vars=['Next token is \' an\'', 'Next token is plausible \' an\'', 'Next token is not \' an\''], 
                        var_name='status', value_name='Proportion of Labels')

In [51]:
grouped_melted[grouped_melted["L31N892 Activation"] == 8]

,L31N892 Activation,status,Proportion of Labels
8,8,Next token is ' an',0.00
28,8,Next token is plausible ' an',0.22
48,8,Next token is not ' an',0.78


In [52]:
fig = px.area(grouped_melted, x='L31N892 Activation', y='Proportion of Labels', color='status', line_shape='spline')
tickvals = np.arange(len(bin_edges) - 1)
ticktext = [f"{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}" for i in tickvals]

fig.update_layout(
    yaxis=dict(range=[0, 1]),
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ),
)
fig.show()

In [60]:
scaled_grouped = grouped_melted

In [82]:
def get_scaled_proportion(row):
    bucket_size = bucket_sizes.iloc[row['L31N892 Activation'], -1]
    scaling_value = bucket_size / total
    return row['Proportion of Labels'] * scaling_value

with open('data/gpt2_large_spectrum_pre_v3_bucket_sizes.pkl', 'rb') as f:
    bucket_sizes = pickle.load(f)
total = bucket_sizes.iloc[:, -1].sum()

merged = scaled_grouped
merged['Scaled Proportion of Labels'] = scaled_grouped.apply(get_scaled_proportion, axis=1)

fig = px.area(merged, x='L31N892 Activation', y='Scaled Proportion of Labels', color='status', line_shape='spline', title='Scaled proportion of labels')

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ))
fig.show()

In [129]:
sample_df['activation']

0     -8.661327
1     -8.225200
2     -8.276969
3     -8.448374
4     -8.197376
         ...   
995    8.312494
996    9.076558
997    8.523770
998    8.845934
999    8.544028
Name: activation, Length: 1000, dtype: float32

In [90]:
print(len(expected_value_df))

60


In [ ]:
post_df = sample_df.copy()
post_df.sort_values(by='L31N892 Activation', inplace=True)
post_df['L31N892 Activation'] = post_df['L31N892 Activation'].rank(method='dense').astype(int) - 1
post_df['post_activation'] = post_df['activation'].apply(lambda x: F.gelu(torch.tensor(x)).item())

post_df = post_df.groupby(['label', 'L31N892 Activation'], as_index=False)['post_activation'].mean()
# post_df.sort_values(by='label', inplace=True, ascending=False)

all_labels = post_df['label'].unique()
all_activations = post_df['L31N892 Activation'].unique()

complete_index = pd.MultiIndex.from_product([all_labels, all_activations], names=['label', 'L31N892 Activation'])
complete_df = pd.DataFrame(index=complete_index).reset_index()

post_mean_acts = pd.merge(complete_df, post_df, how='left', on=['label', 'L31N892 Activation']).fillna(0)

post_mean_acts.sort_values(by=['label', 'L31N892 Activation'], inplace=True, ascending=[False, True])
post_mean_acts


In [137]:
expected_value_df

,L31N892 Activation,status,Proportion of Labels,Scaled Proportion of Labels,activation,Expected Value,post_activation
0,0,Next token is ' an',0.00,0.000000e+00,-8.661327,0.000000e+00,0.000000e+00
1,1,Next token is ' an',0.00,0.000000e+00,-8.225200,0.000000e+00,0.000000e+00
2,2,Next token is ' an',0.00,0.000000e+00,-8.276969,0.000000e+00,0.000000e+00
3,3,Next token is ' an',0.00,0.000000e+00,-8.448374,0.000000e+00,0.000000e+00
4,4,Next token is ' an',0.00,0.000000e+00,-8.197376,0.000000e+00,0.000000e+00
5,5,Next token is ' an',0.00,0.000000e+00,-8.576109,0.000000e+00,0.000000e+00
6,6,Next token is ' an',0.00,0.000000e+00,-8.543550,0.000000e+00,0.000000e+00
7,7,Next token is ' an',0.00,0.000000e+00,-8.441538,0.000000e+00,0.000000e+00
8,8,Next token is ' an',0.00,0.000000e+00,-8.612757,0.000000e+00,0.000000e+00
9,9,Next token is ' an',0.00,0.000000e+00,-8.602530,-0.000000e+00,-2.619242e-08


In [153]:
# Third chart: Activation value * Pr(activation being in bucket)

import torch.nn.functional as F


expected_value_df = scaled_grouped

def get_scaled_proportion(row):
    bucket_size = bucket_sizes.iloc[row['L31N892 Activation'], -1]
    scaling_value = bucket_size / total
    return row['Proportion of Labels'] * scaling_value

with open('data/gpt2_large_spectrum_pre_v3_bucket_sizes.pkl', 'rb') as f:
    bucket_sizes = pickle.load(f)
total = bucket_sizes.iloc[:, -1].sum()

expected_value_df['post_activation'] = post_mean_acts['post_activation']
expected_value_df['Expected Value'] = expected_value_df['post_activation'] * expected_value_df['Scaled Proportion of Labels']


fig = px.area(expected_value_df, x='L31N892 Activation', y='Expected Value', color='status', line_shape='spline')

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ))
fig.show()